In [1]:
import pandas as pd
import numpy as np
import os
import re
import glob
import json

from sentence_transformers import SentenceTransformer

In [4]:
# Specify the path to the folder containing the .md files
folder_path = 'sagemaker_documentation'

# Use glob to find all .md files in the specified folder
md_files = glob.glob(os.path.join(folder_path, '*.md'))

In [5]:
# Create an empty list to store the data
data = []

# Read the contents of each .md file and store in the list
for file in md_files:
    with open(file, 'r', encoding='utf-8') as f:
        content = f.read()
        doc_name = os.path.splitext(os.path.basename(file))[0]  # Get the file name without extension
        data.append({'id': doc_name, 'text': content})

# Convert the list to a DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)

                                                    id  \
0                            amazon-sagemaker-toolkits   
1                    asff-resourcedetails-awssagemaker   
2                automating-sagemaker-with-eventbridge   
3    aws-properties-events-rule-sagemakerpipelinepa...   
4    aws-properties-events-rule-sagemakerpipelinepa...   
..                                                 ...   
331                              sagemaker-rl-workflow   
332                                    sagemaker-roles   
333              services-that-can-integrate-sagemaker   
334     train-evaluate-models-using-sagemaker-notebook   
335                         use-sagemaker-edge-manager   

                                                  text  
0    # Using the SageMaker Training and Inference T...  
1    # AwsSageMaker<a name="asff-resourcedetails-aw...  
2    # Automating Amazon SageMaker with Amazon Even...  
3    # AWS::Events::Rule SageMakerPipelineParameter...  
4    # AWS::Events

In [20]:
# all-MiniLM-L12-v1  
"""
This is a sentence-transformers model: It maps sentences & paragraphs
to a 384 dimensional dense vector space and can be used for tasks like clustering or semantic search.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v1

"""
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v1')
embeddings = model.encode(sentences)
print(embeddings)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\Kiki\anaconda3\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Kiki\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.95k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[-8.15894629e-04  6.87181354e-02  3.49173546e-02  4.14385786e-03
  -7.88317248e-03  3.46956626e-02  6.54209927e-02  4.63761911e-02
   7.13194609e-02  1.39251128e-02  9.75804403e-02 -8.77624284e-03
   5.22356518e-02  1.56827010e-02  4.90205288e-02  8.33300874e-03
   3.52288187e-02 -5.91127165e-02 -1.03482321e-01 -1.59918740e-02
   8.27493817e-02  4.88816760e-02 -4.36305553e-02 -4.90250997e-03
  -4.43353541e-02  4.72674966e-02 -4.54613082e-02  4.95009422e-02
   1.11471124e-01 -4.89814468e-02 -1.39116734e-01 -8.68879166e-03
   4.15085144e-02  3.36105153e-02 -6.77769706e-02  6.99868500e-02
  -1.17844949e-02  3.78630199e-02 -6.58198297e-02 -3.66278691e-03
   3.50759644e-03 -6.73339367e-02  8.49199966e-02 -4.09450270e-02
  -8.46524537e-03 -5.45388460e-02 -2.31756479e-03 -1.36079462e-02
   9.48514417e-02 -2.10678298e-02 -1.33247286e-01 -4.60463613e-02
  -1.02482259e-01 -1.22099649e-02 -9.01510939e-03 -1.70487929e-02
  -2.19867355e-03  4.52472083e-02  1.73924826e-02 -1.10909335e-01
   2.53083

In [25]:
embeddings = []
for index, row in df.iterrows():
    embedding = model.encode(df.iloc[index]['text'])
    print(embedding)
    embeddings.append(embedding)

[-9.82224792e-02 -4.39610630e-02 -5.34540117e-02  4.90738787e-02
  8.19874331e-02 -1.00021930e-02  1.19696902e-02  2.21430641e-02
 -3.64789851e-02 -4.91205230e-02 -9.76334326e-03  9.15431231e-03
  4.47571054e-02 -5.10884002e-02  6.19549192e-02  1.06640361e-01
  2.42777858e-02 -7.92025924e-02  2.56602429e-02 -2.32561398e-02
 -7.06744418e-02 -2.75476798e-02 -2.10241452e-02  1.78021546e-02
 -2.53334157e-02  5.99759668e-02 -4.00311425e-02  7.71726202e-03
  5.37231825e-02  5.03438292e-03  9.77526382e-02 -1.49104064e-02
  5.91301024e-02  1.14782387e-02  1.42156135e-03  1.38295650e-01
 -2.53496552e-03 -8.81710202e-02 -3.55813205e-02  1.23458449e-02
 -1.91704240e-02 -4.45153750e-02 -7.40261599e-02 -2.38392036e-02
 -1.29467938e-02 -2.16767695e-02 -4.69456762e-02 -1.26594806e-03
 -1.05999997e-02  5.65808006e-02 -7.10723922e-02 -9.40730572e-02
 -4.22238372e-02 -4.80823638e-03  1.79724749e-02  6.21942282e-02
 -4.36864011e-02  3.51175740e-02 -2.98984814e-03 -2.48106048e-02
  3.28292064e-02  8.28278

[-4.00842838e-02 -1.49156014e-02 -7.51118138e-02  8.39234330e-03
  3.48787196e-02  2.57430896e-02 -1.27195055e-03  1.99917667e-02
 -3.23722214e-02  4.71784100e-02 -3.48347798e-02 -3.57118212e-02
  5.52404970e-02 -2.18697526e-02  5.30897360e-03  1.10921405e-01
 -7.19030201e-03 -1.08797505e-01 -6.44683018e-02  1.62475761e-02
 -5.05555309e-02 -8.82695541e-02 -5.32750897e-02  2.87113190e-02
 -7.68956989e-02  5.67318983e-02  2.33055241e-02  2.65317149e-02
  5.17938957e-02 -3.62545885e-02  1.95971206e-02 -3.53747644e-02
  5.34759536e-02  7.60655757e-03 -3.12272906e-02  7.55067244e-02
 -5.43509610e-02  2.76426598e-03 -2.62978561e-02 -7.21924230e-02
  8.15460011e-02 -1.31063879e-01 -2.88755316e-02 -3.20833437e-02
  3.66056375e-02 -1.46426810e-02 -1.46928420e-02  4.12826426e-03
 -5.65898744e-03  5.28771915e-02 -1.85290240e-02 -3.22034881e-02
 -5.57154864e-02 -3.02333403e-02 -3.49776037e-02  1.27486959e-01
  7.49829924e-03  2.18485557e-02  1.21717276e-02  1.40863638e-02
  4.80289236e-02 -2.99281

[-5.93225248e-02  3.16594578e-02 -5.76892588e-03 -2.62925238e-03
 -1.61406789e-02  1.53594529e-02  2.29413761e-03  1.30110490e-03
  2.07232740e-02  2.37075742e-02  1.19744968e-02 -9.67935771e-02
  4.62468565e-02 -9.78540033e-02  4.40847464e-02  1.16940796e-01
 -8.85437131e-02 -8.49205777e-02 -8.92896354e-02 -3.33361179e-02
  1.25477770e-02  5.00280894e-02 -7.04264268e-02  7.49548757e-03
 -7.04187974e-02  1.81176793e-02 -3.59789319e-02  6.15785122e-02
  6.75097778e-02 -3.68194804e-02  7.10673854e-02 -6.86896518e-02
  4.00117487e-02  1.49272121e-02  3.94284688e-02  6.88463226e-02
 -4.99698380e-03 -3.18973362e-02  2.32245564e-03 -9.21235885e-04
  6.62377328e-02 -1.38867915e-01 -4.67680320e-02 -1.17080480e-01
  8.32737051e-03 -2.31764838e-02 -3.03154252e-02 -3.78119908e-02
  1.87817290e-02  9.33984518e-02 -6.54712692e-02 -1.13421202e-01
 -5.87999783e-02  5.17634442e-03 -3.34579055e-03  6.38507307e-02
 -7.68477246e-02  1.11808982e-02  9.57066938e-03 -1.00010782e-01
 -5.67233153e-02 -2.90690

[-5.89251965e-02 -4.17764187e-02 -4.48527411e-02  5.78534417e-02
  1.33412316e-01  4.12566178e-02 -2.88052652e-02 -5.65351062e-02
  1.46983704e-02  4.10152227e-02 -3.77716012e-02  8.35454743e-03
  9.90419369e-03 -1.11350574e-01  2.58128922e-02  1.21811688e-01
 -8.33121873e-03 -3.35959382e-02 -4.33884189e-02 -7.47484490e-02
 -1.88633297e-02  1.40814036e-02 -3.51717323e-02  1.38976602e-02
 -5.20423763e-02  2.34848093e-02 -6.17599338e-02  8.77540037e-02
 -9.91984387e-04 -9.24933031e-02  5.71291819e-02 -1.53695876e-02
  3.67818363e-02  3.38668749e-02 -2.27389876e-02  7.20459819e-02
 -4.02547605e-02 -9.03626755e-02 -3.19522340e-03  1.61766596e-02
  3.94490734e-02 -5.60678393e-02 -1.17098931e-02 -7.63168707e-02
 -1.32823121e-02  2.33493019e-02 -3.97930406e-02 -2.77520660e-02
  7.11174756e-02 -5.92191750e-03 -5.61953671e-02 -1.39788851e-01
 -6.80454746e-02 -2.45380886e-02 -1.19853774e-02  3.37416194e-02
 -5.25088683e-02  5.76546043e-02  1.45317838e-02 -3.02992910e-02
  8.52324739e-02 -5.28571

[-4.61160056e-02  3.41901556e-02 -2.19595097e-02  1.63831224e-03
 -5.00850789e-02 -1.47921704e-02  8.44835490e-03 -4.03461716e-04
  2.21503489e-02  3.47367674e-02  1.57310795e-02 -8.49810243e-02
  4.61463481e-02 -1.00311786e-01  4.23574597e-02  1.13667451e-01
 -8.44107792e-02 -6.81405589e-02 -4.27472070e-02 -5.96126504e-02
  1.45720365e-02  7.99409971e-02 -4.68927100e-02  1.35264099e-02
 -5.47969528e-02  3.92196253e-02 -5.60724847e-02  5.50400913e-02
  4.00990918e-02 -4.62881289e-02  6.94605485e-02 -3.80249284e-02
  5.68974055e-02  3.10662575e-02  4.84666824e-02  6.09788336e-02
  1.62997544e-02 -5.06597236e-02 -1.68299433e-02  1.00061623e-02
  5.90366535e-02 -1.64700583e-01 -8.06971639e-02 -1.52046457e-01
 -9.47820395e-03 -7.01683611e-02 -4.37107608e-02 -3.90271954e-02
  1.15713309e-02  7.36779273e-02 -1.03729874e-01 -1.10547192e-01
 -6.51315227e-02  2.28813216e-02 -3.26569043e-02  3.43742445e-02
 -5.16440049e-02  1.14213610e-02 -1.39634963e-02 -1.17987767e-01
 -2.72864904e-02 -1.57732

...

[-1.65857468e-02 -4.05211486e-02 -2.69180443e-02  2.09238827e-02
  7.95866027e-02 -6.40580803e-02 -1.61504932e-02  1.56091135e-02
 -4.27166233e-03 -1.30254654e-02 -6.24468317e-03 -4.70881872e-02
  2.86928955e-02 -8.05735961e-02  3.24577689e-02  7.91862011e-02
  6.57809339e-03 -3.54422592e-02 -2.88079325e-02  1.41657619e-02
 -2.55763568e-02 -8.94034877e-02 -9.87413153e-02  1.19870575e-02
  3.29083041e-03 -5.01194829e-03  5.13843372e-02  1.30799378e-03
 -2.55889315e-02 -4.80912887e-02 -3.34639512e-02  5.48326503e-03
  5.67102656e-02  7.28184283e-02  4.96438928e-02  1.39737859e-01
 -1.43266320e-02 -1.86990295e-02  4.78079356e-02  3.00257131e-02
  7.86903780e-03 -1.35649979e-01 -2.23949868e-02 -7.72309601e-02
  6.48329407e-02  4.77622673e-02 -1.11228943e-01 -1.21237651e-01
 -3.46483141e-02  2.53731571e-02 -6.26285225e-02 -7.62435645e-02
 -5.84919266e-02 -1.89050492e-02 -1.80818103e-02  2.62829792e-02
  4.56955582e-02  3.43022719e-02  1.48628056e-02 -1.48065696e-02
  5.32109924e-02 -2.75378

[-6.72084019e-02 -5.88873886e-02 -8.91918167e-02  4.91449051e-02
  7.30543137e-02  9.24056571e-04 -8.83745253e-02 -3.57497595e-02
  9.21853725e-03 -8.82655755e-03  8.74360837e-03  5.13919853e-02
  5.82541414e-02 -1.03367073e-02  5.81205264e-02  5.06582633e-02
 -1.11076012e-02  7.71995308e-03 -7.31211528e-02 -2.23220401e-02
 -4.57753316e-02 -3.67622897e-02 -4.14745659e-02  7.15860873e-02
 -5.07012568e-02  4.11630347e-02 -6.36651590e-02 -1.13288648e-02
  1.71679575e-02 -5.16286045e-02  8.48223642e-02  4.38979343e-02
  2.16008369e-02  1.00708306e-01 -8.03717691e-03  1.06846362e-01
 -2.93169189e-02 -4.34316471e-02  1.60655621e-02  3.82676646e-02
  5.39069101e-02 -1.67250317e-02 -3.34339328e-02 -1.50290683e-01
  3.70197520e-02 -2.38470789e-02 -1.57568827e-02 -1.58162452e-02
  9.05250316e-04 -2.17757728e-02 -3.33416276e-02 -1.09327130e-01
 -4.86348197e-02  7.69376010e-02 -1.32438252e-02 -3.50555079e-03
 -9.11126751e-03  3.07438951e-02 -2.20339149e-02  2.50369813e-02
  7.58423880e-02 -6.06087

[-6.57935664e-02 -2.78265215e-02 -6.97424412e-02  3.18331495e-02
 -1.93304475e-02  1.46283377e-02  2.59435829e-03  5.57025429e-03
 -5.40116169e-02 -3.47182974e-02 -1.45890201e-02  6.39393851e-02
  9.62762535e-02 -7.26804882e-02  2.37812456e-02  1.01086579e-01
  1.17745558e-02  1.92135256e-02 -7.80702829e-02 -7.83409104e-02
  5.24347126e-02 -8.11105669e-02 -3.26708592e-02  4.68159579e-02
 -5.53799421e-02  2.15419289e-02 -6.15943633e-02  1.16613908e-02
  2.91416030e-02 -6.31906390e-02  4.98793758e-02 -1.23747336e-02
  5.60996197e-02  3.64614709e-04 -2.38364155e-04  1.09356061e-01
 -6.83415681e-02 -4.76389788e-02 -4.38311808e-02  1.55373616e-02
  4.46241200e-02 -6.30394146e-02 -1.20039890e-02 -6.99430704e-02
 -7.64082465e-03 -3.90948914e-02 -4.37297970e-02  1.96742490e-02
  2.56661437e-02  1.03424629e-02 -1.26852453e-01 -8.15851167e-02
 -1.64805562e-03 -8.14912841e-02  1.73276979e-02  1.13354728e-01
  1.79692283e-02  4.88834940e-02 -1.01954080e-02 -6.13548495e-02
 -1.14670387e-02 -4.19676

[-2.26500332e-02 -7.17285089e-03 -8.66003633e-02  1.26360953e-02
  2.24378961e-03 -2.70416029e-02  3.45885530e-02 -1.70013737e-02
 -2.87578125e-02  4.91745397e-02 -7.57074589e-03 -2.57684793e-02
  1.10054843e-01 -2.35641133e-02  5.15593253e-02  1.21566415e-01
  1.38277616e-02 -1.01771094e-01 -1.42050274e-02  3.88040394e-02
 -7.31046945e-02 -6.58300668e-02 -6.49927855e-02 -4.00618679e-04
 -1.13306038e-01  3.08489744e-02 -5.66234291e-02  6.98547363e-02
  3.48063000e-02 -6.76381662e-02  1.11685656e-01 -4.91333269e-02
  6.17948957e-02  1.66878477e-02 -3.64097059e-02  9.83125716e-02
 -1.90132298e-02 -8.61573368e-02 -8.27021804e-03  1.68919917e-02
  8.00761804e-02 -8.58970359e-02 -9.94585305e-02 -9.09784287e-02
  5.67286834e-03 -6.54136017e-02  1.37521382e-02 -1.74801089e-02
  3.05161327e-02  3.34140174e-02 -8.64876285e-02 -6.05673157e-02
 -5.95790669e-02  1.18679814e-02 -2.58519165e-02  5.03206179e-02
 -3.06934882e-02  3.18010077e-02 -5.94113804e-02 -7.85216838e-02
  2.37501878e-02  1.89054

[-1.27708212e-01 -1.73711926e-02 -7.33610541e-02  5.00545278e-02
  4.58786786e-02  4.94143739e-03 -6.73539042e-02  8.96787345e-02
 -1.52934771e-02 -1.32863084e-02 -1.47808669e-03 -3.55500542e-02
  5.15017286e-02 -6.91013411e-02  2.96893194e-02  2.59662494e-02
  7.91927148e-03 -6.12759441e-02 -6.67303652e-02 -3.82840335e-02
 -8.04456621e-02 -3.13669331e-02 -4.35136966e-02  4.69392948e-02
  3.30682122e-03  1.17983920e-02 -4.55902442e-02  1.12330271e-02
  3.72270085e-02 -5.19385114e-02  7.49317333e-02 -3.22791040e-02
  9.67678577e-02  6.44414723e-02 -6.35977611e-02  8.69013816e-02
 -3.19612436e-02 -1.05398588e-01 -5.23196049e-02  1.28914164e-02
 -2.48728991e-02 -8.27578083e-02 -2.21255254e-02 -3.76323909e-02
  4.89983372e-02  1.24175316e-02 -7.31611252e-02  1.09948255e-02
  5.14512807e-02  5.84000275e-02 -8.89944434e-02 -8.53110403e-02
 -4.47364338e-02 -2.69037727e-02 -2.62541156e-02  1.30840212e-01
 -8.55505280e-03  5.38400114e-02 -4.41141762e-02 -3.11328135e-02
  5.05489707e-02 -2.20871

In [ ]:
# https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [26]:
def cosine_similarity(vec1, vec2):
    """
    Calculate the cosine similarity between two vector embeddings.

    Parameters:
    - vec1: A numpy array representing the first vector embedding.
    - vec2: A numpy array representing the second vector embedding.

    Returns:
    - Cosine similarity: A float value between -1 and 1.
    """
    # Ensure the vectors are numpy arrays
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)

    # Compute the dot product between the two vectors
    dot_product = np.dot(vec1, vec2)

    # Compute the L2 norms (magnitudes) of the vectors
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)

    # Calculate the cosine similarity
    cosine_sim = dot_product / (norm_vec1 * norm_vec2)

    return cosine_sim

In [28]:
cosine_similarity(embeddings[0], embeddings[2])

0.56009007

In [29]:
cosine_similarity(embeddings[0], embeddings[3])

0.46280676

In [30]:
cosine_similarity(embeddings[1], embeddings[3])

0.47557846

In [35]:
cosine_similarity(embeddings[31], embeddings[30])

0.6387138